<a href="https://colab.research.google.com/github/ANANAYAGG/minGPT/blob/master/tesseract_unstructured_SIH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1

In [2]:
!apt-get install poppler-utils
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (213 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [3]:
import os

from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [4]:
GROQ_API_KEY = "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"

In [5]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [10]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
loader = DirectoryLoader("/content/sample_data/DATA", glob="./*.txt", loader_cls=UnstructuredPDFLoader)
documents = loader.load()

ERROR:langchain_community.document_loaders.directory:Error loading file /content/sample_data/DATA/Airport_Information.txt


PDFPageCountError: Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table


In [12]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader

loader = DirectoryLoader("/content/sample_data/DATA", glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [13]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

text_chunks = text_splitter.split_documents(documents)

In [14]:
persist_directory = "doc_db"

In [15]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [17]:
retriever = vectorstore.as_retriever()

In [18]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [20]:
query = "TELL ME THE FULL CONCLUSION ABOUT THE AIRPORT?"
response = qa_chain.invoke({"query":query})

In [21]:
print(response)

{'query': 'TELL ME THE FULL CONCLUSION ABOUT THE AIRPORT?', 'result': 'Here is the full conclusion about the Indira Gandhi International Airport:\n\nIndira Gandhi International Airport is a modern and well-equipped airport that offers a seamless travel experience. With its extensive facilities, convenient transportation options, and efficient operations, it serves as a major gateway to India. Whether you are a domestic or international traveler, IGIA provides a comfortable and enjoyable journey.', 'source_documents': [Document(metadata={'source': '/content/sample_data/DATA/Airport_Information.txt'}, page_content="Terminal Information\nTerminal 1: Primarily handles domestic flights and some international flights.\nTerminal 2: Handles international flights, including long-haul flights.\nTerminal 3: Handles international flights, including premium airlines and code-sharing partners.\nAirport Facilities\nRetail and Duty-Free:\nA wide range of shops offering luxury goods, electronics, cosme

In [22]:
print(response["result"])

Here is the full conclusion about the Indira Gandhi International Airport:

Indira Gandhi International Airport is a modern and well-equipped airport that offers a seamless travel experience. With its extensive facilities, convenient transportation options, and efficient operations, it serves as a major gateway to India. Whether you are a domestic or international traveler, IGIA provides a comfortable and enjoyable journey.


In [ ]:
query = "What does the document say about Receptor-binding domain?"
response = qa_chain.invoke({"query":query})
print(response["result"])

The document discusses the Receptor-binding domain (RBD) of the SARS-CoV-2 spike glycoprotein in several sections. Here are the key points:

1. **Location and Structure**: The RBD is located on the S1 subunit of the spike glycoprotein and has a string of domains, including the N-terminal domain (NTD), subdomain 1 (SD1), and the RBD itself. The RBD harbors the ACE2-binding site, which lies across the top of the RBD, spanning the neck and shoulders.

2. **Conformation**: The RBD adopts a range of configurations on the spike, from 'up' to 'down', and only the up conformation can interact with ACE2.

3. **Binding of Neutralizing Antibodies**: The RBD is a major target for neutralizing antibodies, which can be grouped into several clusters based on their epitopes: left shoulder, neck, right shoulder, left flank, and right flank.

4. **Importance of RBD in Neutralization**: Most potent neutralizing antibodies induced by vaccination or natural infection target the RBD and usually interfere wi

In [23]:
!pip install unstructured[local-inference]==0.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.5 MB/s eta 0:00:00


In [26]:
from unstructured.partition.auto import partition
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

def process_and_embed(input_files, persist_directory):
  """Processes a list of files, extracts text, and generates embeddings.

  Args:
      input_files: List of paths to input files.
      persist_directory: Directory to store the embeddings.
  """
  texts = []
  for file in input_files:
    elements = partition(filename=file)
    text = "\n\n".join([str(element) for element in elements])
    texts.append(text)

  embeddings = HuggingFaceEmbeddings()
  vectorstore = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    persist_directory=persist_directory
  )

# Example usage
input_files = ["/content/PNGA.png", "/content/research on NDVI.pdf"]
persist_directory = "embeddings"
process_and_embed(input_files, persist_directory)

yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

def run_query(query, persist_directory):
  """Runs a query on the embedded documents and returns the result.

  Args:
      query: The query to run.
      persist_directory: Directory where the embeddings are stored.
  """
  embeddings = HuggingFaceEmbeddings()
  vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
  )
  retriever = vectorstore.as_retriever()

  llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
  )

  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
  )

  response = qa_chain.invoke({"query": query})
  return response["result"]

# Example usage
query = "GIVE ME BRIEF DESCRIPTION ABOUT PROCESS_AND_EMBED FUNCTION?"
persist_directory = "embeddings"
result = run_query(query, persist_directory)
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The `process_and_embed` function appears to be a part of a larger system that processes a list of files, extracts text from them, and generates embeddings using the Hugging Face Embeddings library.

Here's a brief description of what the function does:

1. It takes two arguments: `input_files` (a list of file paths) and `persist_directory` (a directory to store the generated embeddings).
2. It iterates over each file in the `input_files` list.
3. For each file, it extracts text from the file and joins the extracted text into a single string.
4. It creates an instance of the `HuggingFaceEmbeddings` class to generate embeddings.
5. It uses the `Chroma` library to store the generated embeddings in the specified `persist_directory`.

In summary, the `process_and_embed` function is designed to process a list of files, extract text from them, and generate embeddings that can be stored for later use.
